# Intro

The A&E Synthetic Data.csv is a large dataset; around 4.29GB in size. With this I would only analyze a fraction of the data. 

# Download the DataBase

The raw data could be downloaded in the following link: https://nhsengland-direct-uploads.s3-eu-west-1.amazonaws.com/A%26E+Synthetic+Data.7z?versionId=null

Unzip the file to get the csv. Since the CSV is 4.2GB, I will not be including the file in this repository.

After I downloaded the dataset, I initially used pandas to read the csv but my computer's memory is not up for the task. I searched for an alternative method and I had found the `dask` package to be able to read all the data into a data frame.

In [14]:
# This is the code to install the dask package
# pip install dask

# Importing necessary packages

In [15]:
# import packages
from dask import dataframe as dd
import pandas as pd
import numpy as np

# Read the CSV

The dask dataframe was having trouble in automatically detecting the data types of the different columns of the dataset. So I opened the file in excel just to see the column names and get a feel of the data. I then specified the datypes for each row in the following `read_csv` code.

In [16]:
a_and_e_synth_df = dd.read_csv('A&E Synthetic Data.csv', skiprows=1, dtype={\
    'IMD_Decile_From_LSOA':'object',
    'Age_Band':'object',
    'Sex':'object',
    'AE_Arrive_Date':'object',
    'AE_Arrive_HourOfDay':'object',
    'AE_Time_Mins':'float64',
    'AE_HRG':'object',
    'AE_Num_Diagnoses':'int64',
    'AE_Num_Investigations':'int64',
    'AE_Num_Treatments':'int64',
    'AE_Arrival_Mode':'object',
    'Provider_Patient_Distance_Miles':'float64',
    'ProvID':'object',
    'Admitted_Flag':'object',
    'Admission_Method':'object',
    'ICD10_Chapter_Code':'object',
    'Treatment_Function_Code':'object',
    'Length_Of_Stay_Days':'float64'})
a_and_e_synth_df.head(10)

,IMD_Decile_From_LSOA,Age_Band,Sex,AE_Arrive_Date,AE_Arrive_HourOfDay,AE_Time_Mins,AE_HRG,AE_Num_Diagnoses,AE_Num_Investigations,AE_Num_Treatments,AE_Arrival_Mode,Provider_Patient_Distance_Miles,ProvID,Admitted_Flag,Admission_Method,ICD10_Chapter_Code,Treatment_Function_Code,Length_Of_Stay_Days
0,2,65-84,1,2015-07-02 00:00:00,17-20,210.0,High,1,5,3,2,1.0,15207,1,21,XVIII,180,1.0
1,6,18-24,1,2017-05-31 00:00:00,17-20,20.0,Low,0,1,1,2,5.0,15321,0,NaN,NaN,NaN,NaN
2,2,65-84,2,2015-10-25 00:00:00,13-16,280.0,Nothing,1,0,1,1,2.0,15269,0,NaN,NaN,NaN,NaN
3,4,25-44,1,2014-06-16 00:00:00,21-24,150.0,Low,1,2,2,2,1.0,15239,0,NaN,NaN,NaN,NaN
4,7,18-24,2,2017-11-18 00:00:00,13-16,180.0,Low,1,1,2,2,1.0,15286,0,NaN,NaN,NaN,NaN
5,4,85+,2,2015-09-30 00:00:00,13-16,240.0,Low,1,10,10,1,2.0,15310,1,21,IX,OTHER,16.0
6,8,65-84,2,2015-02-04 00:00:00,09-12,160.0,Nothing,0,1,1,1,2.0,15260,1,21,XVIII,300,1.0
7,2,45-64,2,2015-01-11 00:00:00,09-12,210.0,Low,2,2,1,2,13.0,15312,0,NaN,NaN,NaN,NaN
8,7,1-17,2,2016-11-17 00:00:00,09-12,30.0,Nothing,1,1,1,2,9.0,15396,0,NaN,NaN,NaN,NaN
9,3,65-84,2,2018-01-12 00:00:00,17-20,100.0,Low,1,1,1,2,0.0,15383,0,NaN,NaN,NaN,NaN


# Assess the data

First I will look at the different columns

In [17]:
# get the names of each column
columns=a_and_e_synth_df.columns
columns

Index(['IMD_Decile_From_LSOA', 'Age_Band', 'Sex', 'AE_Arrive_Date',
       'AE_Arrive_HourOfDay', 'AE_Time_Mins', 'AE_HRG', 'AE_Num_Diagnoses',
       'AE_Num_Investigations', 'AE_Num_Treatments', 'AE_Arrival_Mode',
       'Provider_Patient_Distance_Miles', 'ProvID', 'Admitted_Flag',
       'Admission_Method', 'ICD10_Chapter_Code', 'Treatment_Function_Code',
       'Length_Of_Stay_Days'],
      dtype='object')

In [18]:
# iterate for each column and get the description
#for column in columns:
#    print(a_and_e_synth_df[column].describe().compute())

There are 65,836,575 rows in the data set and  it took 11 min to check the description of each of the columns. For this analysis, I will look into the data of all admitted persons.

# Get all admitted patients

In [19]:
# Get all admitted patients
admitted = a_and_e_synth_df[a_and_e_synth_df['Admitted_Flag']=='1']

In [20]:
# Check how many rows available
admitted['IMD_Decile_From_LSOA'].count().compute()

13050415

There is still too much rows. With this I would concentrate on only one provider so I would use the provider that have highest number of entries.

# Get all the entries for the provider with the highest number of entries.

In [21]:
# Check which provider has the highest number of entries
admitted['ProvID'].describe().compute()

unique         238
count     13113928
top          15098
freq        326444
Name: ProvID, dtype: object

In [22]:
# Get all the entries of the provider that has the highest number entries
provider=admitted[admitted['ProvID']=='15098'].compute()
provider

,IMD_Decile_From_LSOA,Age_Band,Sex,AE_Arrive_Date,AE_Arrive_HourOfDay,AE_Time_Mins,AE_HRG,AE_Num_Diagnoses,AE_Num_Investigations,AE_Num_Treatments,AE_Arrival_Mode,Provider_Patient_Distance_Miles,ProvID,Admitted_Flag,Admission_Method,ICD10_Chapter_Code,Treatment_Function_Code,Length_Of_Stay_Days
284,2,1-17,2,2015-05-21 00:00:00,17-20,290.0,Nothing,0,1,2,2,4.0,15098,1,28,XIV,420,1.0
1710,2,65-84,1,2017-09-30 00:00:00,13-16,90.0,Low,1,1,1,2,5.0,15098,1,21,XI,100,0.0
1852,3,1-17,2,2016-09-21 00:00:00,17-20,240.0,Low,0,1,1,2,1.0,15098,1,21,X,420,0.0
1855,1,85+,1,2014-03-23 00:00:00,05-08,240.0,Low,3,8,3,1,2.0,15098,1,21,XI,300,0.0
1909,2,1-17,2,2017-11-11 00:00:00,13-16,130.0,Low,0,10,3,2,4.0,15098,1,21,XVIII,180,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738718,1,1-17,1,2017-11-30 00:00:00,21-24,180.0,Medium,0,8,6,2,2.0,15098,1,21,Other,420,0.0
738790,3,65-84,1,2017-05-12 00:00:00,21-24,240.0,Medium,3,6,7,2,4.0,15098,1,21,Other,300,3.0
738989,3,25-44,2,2015-07-12 00:00:00,01-04,190.0,Medium,1,5,3,1,1.0,15098,1,21,Other,OTHER,1.0
739406,2,45-64,1,2016-08-26 00:00:00,17-20,220.0,Medium,0,10,4,1,168.0,15098,1,21,XVIII,OTHER,43.0


In [24]:
provider.drop(columns=['Admitted_Flag','ProvID'], inplace=True)
provider

,IMD_Decile_From_LSOA,Age_Band,Sex,AE_Arrive_Date,AE_Arrive_HourOfDay,AE_Time_Mins,AE_HRG,AE_Num_Diagnoses,AE_Num_Investigations,AE_Num_Treatments,AE_Arrival_Mode,Provider_Patient_Distance_Miles,Admission_Method,ICD10_Chapter_Code,Treatment_Function_Code,Length_Of_Stay_Days
284,2,1-17,2,2015-05-21 00:00:00,17-20,290.0,Nothing,0,1,2,2,4.0,28,XIV,420,1.0
1710,2,65-84,1,2017-09-30 00:00:00,13-16,90.0,Low,1,1,1,2,5.0,21,XI,100,0.0
1852,3,1-17,2,2016-09-21 00:00:00,17-20,240.0,Low,0,1,1,2,1.0,21,X,420,0.0
1855,1,85+,1,2014-03-23 00:00:00,05-08,240.0,Low,3,8,3,1,2.0,21,XI,300,0.0
1909,2,1-17,2,2017-11-11 00:00:00,13-16,130.0,Low,0,10,3,2,4.0,21,XVIII,180,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738718,1,1-17,1,2017-11-30 00:00:00,21-24,180.0,Medium,0,8,6,2,2.0,21,Other,420,0.0
738790,3,65-84,1,2017-05-12 00:00:00,21-24,240.0,Medium,3,6,7,2,4.0,21,Other,300,3.0
738989,3,25-44,2,2015-07-12 00:00:00,01-04,190.0,Medium,1,5,3,1,1.0,21,Other,OTHER,1.0
739406,2,45-64,1,2016-08-26 00:00:00,17-20,220.0,Medium,0,10,4,1,168.0,21,XVIII,OTHER,43.0


In [ ]:
# Save the dataframe to a csv file
provider.to_csv('AnE_Synthetic_Data_Provider_Admission.csv', index=False)